In [23]:
import torch
import copy
from torch.testing._internal.logging_tensor import LoggingTensorMode, capture_logs


In [33]:
device = "cuda"

hdim = 1024 * 8

main_linear = torch.nn.Linear(hdim, hdim).to(device)

fp16_linear = copy.deepcopy(main_linear).half()

main_inputs = torch.randn(3, 3, hdim).to(device).requires_grad_(True)
fp16_inputs = main_inputs.half().requires_grad_(True)

# main_linear = main_linear.bfloat16()
with capture_logs(is_mode=True) as logs, LoggingTensorMode():

    with torch.autocast(device_type=device, dtype=torch.bfloat16):
        outputs = main_linear(main_inputs)
        loss = outputs[0, 0, 0].float()

    loss.backward()

for l in logs:
    print(l)

# bfloat16 input
with torch.autocast(device_type=device, dtype=torch.float16):
    outputs = fp16_linear(fp16_inputs)
    loss = torch.sum(outputs)

loss.backward()

$1: bf16[8192] = torch._ops.aten._to_copy.default($0, dtype=torch.bfloat16)
$3: bf16[8192, 8192] = torch._ops.aten._to_copy.default($2, dtype=torch.bfloat16)
$5: bf16[3, 3, 8192] = torch._ops.aten._to_copy.default($4, dtype=torch.bfloat16)
$6: bf16[9, 8192] = torch._ops.aten.view.default($5, ['9', '8192'])
$7: bf16[8192, 8192] = torch._ops.aten.t.default($3)
$8: bf16[9, 8192] = torch._ops.aten.addmm.default($1, $6, $7)
$9: bf16[3, 3, 8192] = torch._ops.aten.view.default($8, ['3', '3', '8192'])
$10: bf16[3, 8192] = torch._ops.aten.select.int($9, 0, 0)
$11: bf16[8192] = torch._ops.aten.select.int($10, 0, 0)
$12: bf16[] = torch._ops.aten.select.int($11, 0, 0)
$13: f32[] = torch._ops.aten._to_copy.default($12, dtype=torch.float32)
$14: f32[] = torch._ops.aten.ones_like.default($13, pin_memory=False, memory_format=torch.preserve_format)
$15: bf16[] = torch._ops.aten._to_copy.default($14, dtype=torch.bfloat16, layout=torch.strided, device=device(type='cuda', index=0))
$16: bf16[8192] = torch

In [20]:
print(torch.equal(fp16_linear.weight.grad, main_linear.weight.grad.half()))
print(torch.equal(fp16_linear.weight.grad.float(), main_linear.weight.grad))

print(torch.equal(fp16_linear.weight.grad, main_linear.weight.grad.half()))
print(torch.equal(fp16_linear.weight.grad.float(), main_linear.weight.grad))

True
True
True
True
